#Problem 1(a).  A collectible card class.

You are tasked with organizing information about a Magic: The Gathering card collection.  You are particularly interested in which cards are most rare, and therefore most valuable.

The MTG JSON website (http://mtgjson.com/) has information about different sets of Magic: The Gathering cards in JSON format.  Using the [Example Card](http://mtgjson.com/#exampleCard) information, create a MagicCard class to store relevant information about Magic cards in Python.

Consider:

* How should someone initialize an instance of your class?
* What functions should your class have?

In [1]:
import json
import itertools
import pandas as pd
from collections import Counter

class Card(object):
    '''A Magic The Gathering Card Class'''
    def __init__(self, id_, name, rarity, colors):
        self.__id = id_
        self.__name = name
        self.__rarity = rarity
        self.__colors = colors
    
    def __repr__(self):
        return "Card(id_='{0}', name='{1}', rarity='{2}', colors='{3}')".format( 
                self.__id_, 
                self.__name, 
                self.__rarity, 
                self.__colors)

    def __str__(self):
        return "Id {0}, Name: {1}, Rarity: {2}, Colors: {3}".format(
                self.__id_, 
                self.__name, 
                self.__rarity, 
                self.__colors)

    def get_id(self):
        return self.__id

    def get_name(self):
        return self.__name

    def get_rarity(self):
        return self.__rarity

    def get_colors(self):
        return self.__colors

#Problem 1(b). Class for a set of Magic cards.

Download the data for the Dragons of Tarkir set from the [Individual Sets](http://mtgjson.com/#individualSets) section of the MTG JSON page.  Create a MagicCardSet class to store relevant information about a set of Magic cards, and create an instance of the class which contains all of the information about the Dragons of Tarkir set.

In [14]:
class CardSet(object):
    '''A Magic The Gathering Card Set Class'''
    def __init__(self, mkm_id, mkm_name, type_, releaseDate, cards):
        self.__mkm_id = mkm_id 
        self.__mkm_name = mkm_name
        self.__type = type_
        self.__releaseDate = releaseDate
        self.__cards = cards

    def __repr__(self):
        return "CardSet(mkm_id='{0}', mkm_name='{1}', type='{2}', releaseDate='{3}', cards='{4}')".format(
                self.__mkm_id, 
                self.__mkm_name,
                self.__type,
                self.__releaseDate,
                self.__cards)
        
    def __str__(self):
        return "mkm_id: {0}, mkm_name: {1}, type: {2}, releaseDate: {3}, numCards: {4}".format( 
                self.__mkm_id, 
                self.__mkm_name,
                self.__type,
                self.__releaseDate,
                len(self.__cards))

    def get_rarest(self):
        return [x for x in self.__cards if x.get_rarity() == 'Mythic Rare']

def create_from_json(json_file):
    with open(json_file, 'r', encoding='utf-8') as fobj:
        data = json.loads(fobj.read())

    card_list = [Card(card.get('id'), 
                      card.get('name'), 
                      card.get('rarity'),
                      card.get('colors')) 
                  for card in data.get('cards')]    

    return CardSet(data.get('mkm_id'), 
                   data.get('mkm_name'), 
                   data.get('type'), 
                   data.get('releaseDate'), 
                   card_list)

cs = create_from_json('DTK-x.json')
print(cs)

mkm_id: 1601, mkm_name: Dragons of Tarkir, type: expansion, releaseDate: 2015-03-27, numCards: 264


#Problem 1(c).  Dragons of Tarkir.

What are the rarest cards in the Dragons of Tarkir set?  What colors are they associated with?

Write Python code to answer this question, then create a markdown cell which explains your method and conclusions.

In [19]:
rare = cs.get_rarest()

df = pd.DataFrame([(x.get_id(), 
                    x.get_name(), 
                    x.get_rarity(), 
                    x.get_colors()) for x in rare],
                  columns = ['id', 'name', 'rarity', 'colors'])

c = Counter(list(itertools.chain.from_iterable(df.colors)))
print(df.name.sort_values())
print(c.most_common(3))

1                 Clone Legion
6             Deathmist Raptor
4       Descent of the Dragons
5             Dragon Whisperer
8            Dragonlord Atarka
9           Dragonlord Dromoka
10         Dragonlord Kolaghan
11           Dragonlord Ojutai
12         Dragonlord Silumgar
13         Narset Transcendent
0             Ojutai Exemplars
3            Risen Executioner
14            Sarkhan Unbroken
7     Shaman of Forgotten Ways
2       Shorecrasher Elemental
Name: name, dtype: object
[('Blue', 6), ('Red', 5), ('Green', 5)]


First, I selected all cards with rarity of "Mythic Rare". This is the most rare a card can be. The method to return rare cards is part of the CardSet class and returns a list of Cards. After, I created a dataframe from this list by creating tuples of the attributes of the Card class using the appropriate get method. Finally, I used a Counter class to count the most common colors. This was slightly challenging because each card could have multiple colors. As such, I needed to flatten the list of lists. For that, I used the itertools.chain. 

There are 15 Mythic Rare cards in the Dragons of Tarkir set. The most common color appears to be Blue, followed by Red and Green.